In [2]:
import pandas as pd
import numpy as np


# Read the csv file
df = pd.read_csv('_Data/Original_dataset/preprocessed_data.csv', header=0, index_col=0)
df.reset_index(drop=True,inplace=True)

# Set tables reading options
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# Create a copy of dataframe
new_df = df.copy(deep=True)

# Create new features "best_of" and "minutes" for players
new_df['PlayerA_bestof'] = np.nan
new_df['PlayerB_bestof'] = np.nan
new_df['PlayerA_minutes'] = np.nan
new_df['PlayerB_minutes'] = np.nan

# Create new features for percentage of winning
new_df['PlayerA_Win%'] = np.nan
new_df['PlayerB_Win%'] = np.nan
new_df

,PlayerA_name,PlayerB_name,Year,Day,best_of,draw_size,round,minutes,PlayerA_id,PlayerB_id,PlayerA_FR,PlayerB_FR,PlayerA_righthanded,PlayerB_righthanded,PlayerA_age,PlayerA_rank,PlayerA_rank_points,PlayerA_svpt%,PlayerA_1st_serve%,PlayerA_1st_serve_won%,PlayerA_2nd_serve_won%,PlayerA_ace%,PlayerA_df%,PlayerA_bp_faced%,PlayerA_bp_saved%,PlayerB_age,PlayerB_rank,PlayerB_rank_points,PlayerB_svpt%,PlayerB_1st_serve%,PlayerB_1st_serve_won%,PlayerB_2nd_serve_won%,PlayerB_ace%,PlayerB_df%,PlayerB_bp_faced%,PlayerB_bp_saved%,surface_Carpet,surface_Clay,surface_Grass,surface_Hard,PlayerA_Win,PlayerA_bestof,PlayerB_bestof,PlayerA_minutes,PlayerB_minutes,PlayerA_Win%,PlayerB_Win%
0,Sergi Bruguera,Tomas Anzari,1993.0,212.0,3.0,32.0,32.0,67.0,101843.0,101765.0,0.0,0.0,1.0,1.0,22.543463,5.0,2531.0,0.472727,0.469388,0.652174,0.615385,0.019231,0.038462,0.096154,0.600000,23.107460,310.0,86.0,0.527273,0.629630,0.470588,0.350000,0.034483,0.034483,0.189655,0.454545,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Jiri Novak,Christian Miniussi,1993.0,212.0,3.0,32.0,32.0,163.0,102562.0,101368.0,0.0,0.0,1.0,1.0,18.365503,584.0,25.0,0.513308,0.680000,0.552941,0.375000,0.022222,0.051852,0.125926,0.470588,26.078030,213.0,168.0,0.486692,0.512605,0.622951,0.379310,0.007812,0.062500,0.132812,0.588235,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Jordi Arrese,Alex Corretja,1993.0,212.0,3.0,32.0,32.0,105.0,101063.0,102374.0,0.0,0.0,1.0,1.0,28.925394,90.0,484.0,0.486301,0.626866,0.595238,0.520000,0.014085,0.042254,0.084507,0.333333,19.310062,66.0,572.0,0.513699,0.391304,0.666667,0.309524,0.026667,0.053333,0.160000,0.500000,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Jaime Oncins,Magnus Larsson,1993.0,212.0,3.0,32.0,32.0,74.0,101761.0,101723.0,0.0,0.0,1.0,1.0,23.129364,80.0,520.0,0.495327,0.326531,0.812500,0.575758,0.000000,0.075472,0.056604,0.666667,23.356606,61.0,622.0,0.504673,0.460000,0.565217,0.370370,0.055556,0.018519,0.111111,0.166667,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Andrei Cherkasov,Bohdan Ulihrach,1993.0,212.0,3.0,32.0,32.0,82.0,101772.0,102539.0,0.0,0.0,1.0,1.0,23.080082,53.0,660.0,0.452381,0.490566,0.846154,0.555556,0.052632,0.017544,0.052632,0.666667,18.439425,620.0,21.0,0.547619,0.419355,0.653846,0.388889,0.014493,0.086957,0.086957,0.166667,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,Slava Dosedel,Simon Youl,1993.0,212.0,3.0,32.0,32.0,76.0,101792.0,101154.0,0.0,0.0,1.0,1.0,22.967831,99.0,428.0,0.540541,0.407407,0.727273,0.718750,0.083333,0.016667,0.000000,0.000000,28.087612,206.0,172.0,0.459459,0.562500,0.703704,0.476190,0.039216,0.019608,0.078431,0.250000,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
6,Carlos Costa,David Rikl,1993.0,212.0,3.0,32.0,32.0,60.0,101475.0,101866.0,0.0,0.0,1.0,0.0,25.278576,35.0,871.0,0.468085,0.425000,0.764706,0.695652,0.045455,0.045455,0.000000,0.000000,22.428474,113.0,357.0,0.531915,0.469388,0.565217,0.346154,0.000000,0.020000,0.180000,0.444444,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
7,Tomas Nydahl,Richard Fromberg,1993.0,212.0,3.0,32.0,32.0,77.0,101459.0,101735.0,0.0,0.0,1.0,1.0,25.366188,186.0,211.0,0.490385,0.625000,0.766667,0.611111,0.019608,0.039216,0.000000,0.000000,23.263517,42.0,796.0,0.509615,0.489362,0.608696,0.375000,0.037736,0.075472,0.113208,0.333333,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
8,Paul Haarhuis,Martin Damm,1993.0,212.0,3.0,32.0,32.0,84.0,101235.0,102093.0,0.0,0.0,1.0,1.0,27.449692,52.0,660.0,0.556452,0.500000,0.843750,0.500000,0.057971,0.014493,0.072464,0.800000,21.002054,102.0,407.0,0.443548,0.469388,0.521739,0.538462,0.054545,0.054545,0.072727,0.000000,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
9,Gerard Solves,David Prinosil,1993.0,212.0,3.0,32.0,32.0,123.0,101468.0,102184.0,1.0,0.0,1.0,1.0,25.319645,182.0,216.0,0.476190,0.638095,0.641791,0.421053,0.018182,0.027273,0.127273,0.642857,20.399727,73.0,545.0,0.523810,0.471154,0.653061,0.363636,0.041322,0.099174,0.123967,0.466667,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Weights of surface weighting
corr_df = pd.read_csv('_Data/New_stats_dataset/correlation_between_surfaces.csv', header=0, index_col=0)

# Columns of the players' stats
playerA_cols = [2,3,4,7] + list(range(17,25)) + [36,37,38,39]
playerB_cols = [2,3,4,7] + list(range(28,36)) + [36,37,38,39]

# FOR EACH MATCH OF DATAFRAME
for i, match in df.iterrows():
    print(i)
    
    # Get the current date of the match
    curr_year = match['Year']
    curr_day = match['Day']
    
    # Get the surfaces weights depending on the surface of the match
    if (match['surface_Carpet'] == 1):
        weight_carpet = corr_df.loc['Carpet','Carpet']
        weight_grass = corr_df.loc['Carpet','Grass']
        weight_hard = corr_df.loc['Carpet','Hard']
        weight_clay = corr_df.loc['Carpet','Clay']
    elif (match['surface_Clay'] == 1):
        weight_carpet = corr_df.loc['Clay','Carpet']
        weight_grass = corr_df.loc['Clay','Grass']
        weight_hard = corr_df.loc['Clay','Hard']
        weight_clay = corr_df.loc['Clay','Clay']
    elif (match['surface_Grass'] == 1):
        weight_carpet = corr_df.loc['Grass','Carpet']
        weight_grass = corr_df.loc['Grass','Grass']
        weight_hard = corr_df.loc['Grass','Hard']
        weight_clay = corr_df.loc['Grass','Clay']
    elif (match['surface_Hard'] == 1):
        weight_carpet = corr_df.loc['Hard','Carpet']
        weight_grass = corr_df.loc['Hard','Grass']
        weight_hard = corr_df.loc['Hard','Hard']
        weight_clay = corr_df.loc['Hard','Clay']

    # Take all past matches of PLAYER 1 and look for same id in playerA and playerB
    id_1 = match['PlayerA_id']
    p1_playerA_rows = df.index[(df['PlayerA_id'] == id_1) & (df['Year'] + df['Day']/365 < curr_year + curr_day/365)].tolist()
    p1_playerB_rows = df.index[(df['PlayerB_id'] == id_1) & (df['Year'] + df['Day']/365 < curr_year + curr_day/365)].tolist()
    p1_playerA_df = df.iloc[p1_playerA_rows, playerA_cols]
    p1_playerA_df['Win'] = 1
    p1_playerB_df = df.iloc[p1_playerB_rows, playerB_cols]
    p1_playerB_df['Win'] = 0
    p1_playerB_df.columns = list(p1_playerA_df)
    tmp1_df = pd.concat([p1_playerA_df,p1_playerB_df], ignore_index=True)
    
    # Take all past matches of PLAYER 2 and look for same id in playerA and playerB
    id_2 = match['PlayerB_id']
    p2_playerA_rows = df.index[(df['PlayerA_id'] == id_2) & (df['Year'] + df['Day']/365 < curr_year + curr_day/365)].tolist()
    p2_playerB_rows = df.index[(df['PlayerB_id'] == id_2) & (df['Year'] + df['Day']/365 < curr_year + curr_day/365)].tolist()
    p2_playerA_df = df.iloc[p2_playerA_rows, playerA_cols]
    p2_playerA_df['Win'] = 1
    p2_playerB_df = df.iloc[p2_playerB_rows, playerB_cols]
    p2_playerB_df['Win'] = 0
    p2_playerA_df.columns = list(p2_playerB_df)
    tmp2_df = pd.concat([p2_playerA_df,p2_playerB_df], ignore_index=True)
    
    # Check if players have played enough matches to compute representative stats
    #if (tmp1_df.empty) | (tmp2_df.empty):
    if (len(tmp1_df) < 20) | (len(tmp2_df) < 20):
        continue

    # Compute a weight for each past match of PLAYER 1
    tmp1_df['elapsing_time'] = (curr_year + curr_day/365) - (tmp1_df['Year'] + tmp1_df['Day']/365)
    tmp1_df['weight'] = tmp1_df['elapsing_time'].apply(lambda t: 0.6**t)
    tmp1_df.loc[tmp1_df['elapsing_time'] <= 1, 'weight'] = 1
    tmp1_df['weight'] = (0.95 * tmp1_df['weight']) + (0.05 * (weight_carpet*tmp1_df['surface_Carpet'] + weight_clay*tmp1_df['surface_Clay'] + weight_grass*tmp1_df['surface_Grass'] + weight_hard*tmp1_df['surface_Hard']))
    tmp1_df.drop(columns=['Year', 'Day', 'elapsing_time', 'surface_Carpet', 'surface_Clay', 'surface_Grass', 'surface_Hard'], inplace = True)

    # Compute a weight for each past match of PLAYER 2
    tmp2_df['elapsing_time'] = (curr_year + curr_day/365) - (tmp2_df['Year'] + tmp2_df['Day']/365)
    tmp2_df['weight'] = tmp2_df['elapsing_time'].apply(lambda t: 0.6**t)
    tmp2_df.loc[tmp2_df['elapsing_time'] <= 1, 'weight'] = 1
    tmp2_df['weight'] = (0.95 * tmp2_df['weight']) + (0.05 * (weight_carpet*tmp2_df['surface_Carpet'] + weight_clay*tmp2_df['surface_Clay'] + weight_grass*tmp2_df['surface_Grass'] + weight_hard*tmp2_df['surface_Hard']))
    tmp2_df.drop(columns=['Year', 'Day', 'elapsing_time', 'surface_Carpet', 'surface_Clay', 'surface_Grass', 'surface_Hard'], inplace = True)

    # Compute the weighted average of PLAYER 1
    weighted_means = np.average(tmp1_df, weights=tmp1_df['weight'],axis=0)
    weighted1_df = pd.DataFrame(weighted_means.reshape(-1, len(weighted_means)), columns=list(tmp1_df.columns))
    weighted1_df = weighted1_df.drop('weight', axis=1)

    # Compute the weighted average of PLAYER 2
    weighted_means = np.average(tmp2_df, weights=tmp2_df['weight'],axis=0)
    weighted2_df = pd.DataFrame(weighted_means.reshape(-1, len(weighted_means)), columns=list(tmp2_df.columns))
    weighted2_df = weighted2_df.drop('weight', axis=1)

    # Add stats of of PLAYER 1 in new dataframe
    new_df.at[i, 17:25] = weighted1_df.iloc[0, 2:10]
    new_df.at[i, 'PlayerA_bestof'] = weighted1_df['best_of']
    new_df.at[i, 'PlayerA_minutes'] = weighted1_df['minutes']
    new_df.at[i, 'PlayerA_Win%'] = weighted1_df['Win']

    # Add stats of of PLAYER 1 in new dataframe
    new_df.at[i, 28:36] = weighted2_df.iloc[0, 2:10]
    new_df.at[i, 'PlayerB_bestof'] = weighted2_df['best_of']
    new_df.at[i, 'PlayerB_minutes'] = weighted2_df['minutes']
    new_df.at[i, 'PlayerB_Win%'] = weighted2_df['Win']

# Drop matches with too less stats
new_df = new_df.dropna()
new_df.reset_index(drop=True, inplace=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702


In [ ]:
# Drop "minutes" feature (won't know that for predicting matches)
new_df.drop(columns=['minutes'], inplace=True)
new_df

In [ ]:
# Reorder columns and save
cols = ['PlayerA_name',
        'PlayerB_name',
        'Year',
        'Day',
        'best_of',
        'draw_size',
        'round',
        'PlayerA_id',
        'PlayerB_id',
        'PlayerA_FR',
        'PlayerB_FR',
        'PlayerA_righthanded',
        'PlayerB_righthanded',
         'PlayerA_age',
         'PlayerA_rank',
         'PlayerA_rank_points',
        'PlayerA_Win%',
        'PlayerA_bestof',
        'PlayerA_minutes',
        'PlayerA_svpt%',
        'PlayerA_1st_serve%',
        'PlayerA_1st_serve_won%',
        'PlayerA_2nd_serve_won%',
        'PlayerA_ace%',
        'PlayerA_df%',
        'PlayerA_bp_faced%',
        'PlayerA_bp_saved%',
         'PlayerB_age',
         'PlayerB_rank',
         'PlayerB_rank_points',
        'PlayerB_Win%',
        'PlayerB_bestof',
        'PlayerB_minutes',
         'PlayerB_svpt%',
        'PlayerB_1st_serve%',
        'PlayerB_1st_serve_won%',
        'PlayerB_2nd_serve_won%',
        'PlayerB_ace%',
        'PlayerB_df%',
        'PlayerB_bp_faced%',
        'PlayerB_bp_saved%',
        'surface_Carpet',
        'surface_Clay',
        'surface_Grass',
        'surface_Hard',
        'PlayerA_Win']
new_df = new_df[cols]

# Save dataset
new_df.to_csv('_Data/New_stats_dataset/new_stats_data_weight06_+surface_weighting_min20matches.csv', sep=',', encoding='utf-8', float_format='%.10f', decimal='.')
new_df